In [1]:
from __future__ import print_function, division

%matplotlib inline
%precision 3

import warnings
warnings.filterwarnings('ignore')

import random
import networkx as nx
import numpy as np
import scipy.stats as stats

import thinkplot
from thinkstats2 import Pmf, Cdf
import thinkcomplexity

In [2]:
def generate_nodes(G):
    """ Generates all the nodes in G """
    for node in G:
        yield node

def sample_nodes(G, n=1000):
    """ Generates n random nodes """
    nodes = G.nodes()
    for i in range(n):
        node = np.random.choice(nodes)
        yield node
        
def compare_node_degree(G):
    """ 
    Plots the distribution of degrees for all the nodes 
    (from generate_nodes) and for a random sampling of 
    nodes (from sample_nodes)
    """
    # enumerate all the nodes
    node_degree = [G.degree(node) for node in generate_nodes(G)]
    thinkplot.Cdf(Cdf(node_degree), label='generate_nodes')

    # generate a random sample of nodes
    node_degree_sample = [G.degree(node) for node in sample_nodes(G)]
    thinkplot.Cdf(Cdf(node_degree_sample), label='sample_nodes')
    
    thinkplot.Config(xlabel='degree', ylabel='CDF')      

In [3]:
def generate_friends(G):
    """
    Generates all friends by iterating through all nodes and
    yielding each of their friends in succession
    """
    for node in G:
        for friend in G[node]:
            yield friend
            
def sample_friends(G, n=1000):
    """
    Generates n random friends by picking a random node then
    randomly picking one of their friends
    """
    nodes = G.nodes()
    for _ in range(n):
        node = np.random.choice(nodes)
        friends = G.neighbors(node)
        friend = np.random.choice(friends)
        yield friend
        
def sample_edges(G, n=1000):
    """
    Generates n random friends by randomly choosing an edge
    and randomly picking a node on either side
    """
    edges = G.edges()
    for _ in range(n):
        # NOTE: you can't use np.random.choice to choose
        # from edges, because it treats a list of pairs
        # as an array with two columns
        edge = random.choice(edges)
        yield random.choice(edge)
        
def compare_friend_degree(G):
    """
    Plots the degrees of all nodes vs. the degrees of all friends
    vs. the degrees of the sampling of friends by node vs. the
    degrees of the sampling of friends by edges.
    """
    
    # enumerate the nodes
    node_degree = [G.degree(node) for node in generate_nodes(G)]
    thinkplot.Cdf(Cdf(node_degree), color='gray')
    
    # enumerate the friends
    friend_degree = [G.degree(node) for node in generate_friends(G)]
    thinkplot.Cdf(Cdf(friend_degree), label='generate_friends')

    # sample friends
    friend_degree_sample = [G.degree(node) for node in sample_friends(G)]
    thinkplot.Cdf(Cdf(friend_degree_sample), color='green', label='sample_friends')
    
    # sample edges
    edge_degree_sample = [G.degree(node) for node in sample_edges(G)]
    thinkplot.Cdf(Cdf(edge_degree_sample), color='red', label='sample_edges')
    
    thinkplot.Config(xlabel='degree', ylabel='CDF')

In [5]:
# graphs

def read_graph(filename):
    G = nx.Graph()
    array = np.loadtxt(filename, dtype=int)
    G.add_edges_from(array)
    return G

ba = nx.barabasi_albert_graph(n=4000, m=20)
ws = nx.watts_strogatz_graph(40000, 50, p=0.4)
fb = read_graph('facebook_combined.txt.gz')
enron = read_graph("email-Enron.txt.gz")
youtube = read_graph("com-youtube.ungraph.txt.gz")

Degree Associativity

In [6]:
def get_degree_pairs_edge_sample_u(G):
    res = []
    for u, v in G.edges_iter():
        res.append((G.degree(u), G.degree(v)))    
    return np.array(res).transpose()

def get_degree_pairs_edge_sample_v(G):
    res = []
    for u, v in G.edges_iter():
        res.append((G.degree(v), G.degree(u)))
    return np.array(res).transpose()

def get_degree_pairs_node_sample(G):
    res = []
    for u, deg_u in G.degree_iter():
        for neighbor in G[u]:
            res.append((G.degree(u), G.degree(neighbor)))
    return np.array(res).transpose()

In [10]:
# degree associativity
# the likelihood that a node is connected to nodes that are similar in degree,
# from -1 (very different) to 1 (similar)
# the facebook paper has a value for their graph at r = 0.226

ba_node_sample = nx.degree_pearson_correlation_coefficient(ba)
ba_degree_pairs_u = get_degree_pairs_edge_sample_u(ba)
ba_degree_pairs_v = get_degree_pairs_edge_sample_v(ba)
ba_edge_sample_u =  np.corrcoef(ba_degree_pairs_u)[0,1]
ba_edge_sample_v =  np.corrcoef(ba_degree_pairs_v)[0,1]

fb_node_sample = nx.degree_pearson_correlation_coefficient(fb)
fb_degree_pairs_u = get_degree_pairs_edge_sample_u(fb)
fb_degree_pairs_v = get_degree_pairs_edge_sample_v(fb)
fb_edge_sample_u =  np.corrcoef(fb_degree_pairs_u)[0,1]
fb_edge_sample_v =  np.corrcoef(fb_degree_pairs_v)[0,1]

ws_node_sample = nx.degree_pearson_correlation_coefficient(ws)
ws_degree_pairs_u = get_degree_pairs_edge_sample_u(ws)
ws_degree_pairs_v = get_degree_pairs_edge_sample_v(ws)
ws_edge_sample_u = np.corrcoef(ws_degree_pairs_u)[0,1]
ws_edge_sample_v = np.corrcoef(ws_degree_pairs_v)[0,1]

enron_node_sample = nx.degree_pearson_correlation_coefficient(enron)
enron_degree_pairs_u = get_degree_pairs_edge_sample_u(enron)
enron_degree_pairs_v = get_degree_pairs_edge_sample_v(enron)
enron_edge_sample_u =  np.corrcoef(enron_degree_pairs_u)[0,1]
enron_edge_sample_v =  np.corrcoef(enron_degree_pairs_v)[0,1]

yt_node_sample = nx.degree_pearson_correlation_coefficient(youtube)
yt_degree_pairs_u = get_degree_pairs_edge_sample_u(youtube)
yt_degree_pairs_v = get_degree_pairs_edge_sample_v(youtube)
yt_edge_sample_u =  np.corrcoef(yt_degree_pairs_u)[0,1]
yt_edge_sample_v =  np.corrcoef(yt_degree_pairs_v)[0,1]

In [16]:
#table degree assoc.

print("| {:20s} | {:20s} | {:20s} | {:20s} | {:20s} |"
      .format("Network", "Node sampled", "Edge sampled (u)", "Edge sampled (v)", "Ratio (edge / node)"))
print("| {:20s} | {:20s} | {:20s} | {:20s} | {:20s} |"
      .format("---", "---", "---", "---", "---"))
print("| {:20s} | {:20f} | {:20f} | {:20f} | {:20f} |"
      .format("Barabasi-Albert", ba_node_sample, ba_edge_sample_u, ba_edge_sample_v, 
              ba_edge_sample_u / ba_node_sample))
print("| {:20s} | {:20f} | {:20f} | {:20f} | {:20f} |"
      .format("Watts-Strogatz", ws_node_sample, ws_edge_sample_u, ws_edge_sample_v, 
              ws_edge_sample_u / ws_node_sample))
print("| {:20s} | {:20f} | {:20f} | {:20f} | {:20f} |"
      .format("Facebook", fb_node_sample, fb_edge_sample_u, fb_edge_sample_v, 
              fb_edge_sample_u / fb_node_sample))
print("| {:20s} | {:20f} | {:20f} | {:20f} | {:20f} |"
      .format("Enron", enron_node_sample, enron_edge_sample_u, enron_edge_sample_v, 
              enron_edge_sample_u / enron_node_sample))
print("| {:20s} | {:20f} | {:20f} | {:20f} | {:20f} |"
      .format("Youtube", yt_node_sample, yt_edge_sample_u, yt_edge_sample_v, 
              yt_edge_sample_u / yt_node_sample))

| Network              | Node sampled         | Edge sampled (u)     | Edge sampled (v)     | Ratio (edge / node)  |
| ---                  | ---                  | ---                  | ---                  | ---                  |
| Barabasi-Albert      |            -0.010436 |             0.347216 |             0.347216 |           -33.270468 |
| Watts-Strogatz       |            -0.006807 |            -0.005890 |            -0.005890 |             0.865321 |
| Facebook             |             0.063577 |             0.114743 |             0.114743 |             1.804775 |
| Enron                |            -0.110764 |             0.024638 |             0.024638 |            -0.222432 |
| Youtube              |            -0.036910 |            -0.025486 |            -0.025486 |             0.690500 |


In [19]:
def degeneracy(G):
    max_degree = max([len(G[node]) for node in generate_nodes(G)])
    return max_degree

print("| {:15s} | {:15s} | {:15s} | {:15s} | {:15s} |"
      .format("Network", "Number of Nodes", "Number of Edges", "Cluster. Coef.", "Degeneracy"))
print("| {:15s} | {:15s} | {:15s} | {:15s} | {:15s} |"
      .format("---", "---", "---", "---", "---"))
print("| {:15s} | {:15d} | {:15d} | {:15f} | {:15f} |"
      .format("Barabasi-Albert", len(ba.nodes()), len(ba.edges()), thinkcomplexity.clustering_coefficient(ba), degeneracy(ba)))
print("| {:15s} | {:15d} | {:15d} | {:15f} | {:15f} |"
      .format("Watts-Strogatz", len(ws.nodes()), len(ws.edges()), thinkcomplexity.clustering_coefficient(ws), degeneracy(ws)))
print("| {:15s} | {:15d} | {:15d} | {:15f} | {:15f} |"
      .format("Facebook", len(fb.nodes()), len(fb.edges()), thinkcomplexity.clustering_coefficient(fb), degeneracy(fb)))
print("| {:15s} | {:15d} | {:15d} | {:15f} | {:15f} |"
      .format("Enron", len(enron.nodes()), len(enron.edges()), thinkcomplexity.clustering_coefficient(enron), degeneracy(enron)))
print("| {:15s} | {:15d} | {:15d} | {:15f} | {:15f} "
      .format("Youtube", len(youtube.nodes()), len(youtube.edges()), thinkcomplexity.clustering_coefficient(youtube), degeneracy(youtube)))

| Network         | Number of Nodes | Number of Edges | Cluster. Coef.  | Degeneracy      |
| ---             | ---             | ---             | ---             | ---             |
| Barabasi-Albert |            4000 |           79600 |        0.035609 |      499.000000 |
| Watts-Strogatz  |           40000 |         1000000 |        0.159371 |       68.000000 |
| Facebook        |            4039 |           88234 |        0.605547 |     1045.000000 |
| Enron           |           36692 |          183831 |        0.496983 |     1383.000000 |
| Youtube         |         1134890 |         2987624 |        0.080802 |    28754.000000 
